In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from format_the_data import format_mi_band_data

In [2]:
### set os dir to data
os.chdir('/Users/Gerard/Downloads')
andy = pd.read_csv('20231030_8210796956_MiFitness_hlth_center_fitness_data.csv')
tom = pd.read_csv('20231030_8211531339_MiFitness_hlth_center_fitness_data.csv')
huub = pd.read_csv('20231031_8210564343_MiFitness_hlth_center_fitness_data.csv')
oumaima = pd.read_csv('20231101_8210586841_MiFitness_hlth_center_fitness_data.csv')

behaviour_tracking_data = pd.read_csv('Data app.csv')

In [3]:
andy['Person'] = 'andy'
tom['Person'] = 'tom'
huub['Person'] = 'huub'
oumaima['Person'] = 'oumaima'

master_frame = pd.concat([andy, tom, huub, oumaima], ignore_index=True)

In [5]:
unique_keys = ['pai',
               'valid_stand', 
               'calories',
               'steps',
               'heart_rate',
               'intensity',
               'dynamic',
               'single_heart_rate',
               'single_spo2',
               'training_load',
               'single_stress',
               'stress',
               'watch_night_sleep',
               'resting_heart_rate',
               'watch_daytime_sleep',
               'weight']
key_dataframes = format_mi_band_data(unique_keys, master_frame)


# separate dataframes:
pai_df = key_dataframes['pai']
valid_stand_df = key_dataframes['valid_stand']
calories_df = key_dataframes['calories']
steps_df = key_dataframes['steps']
heart_rate_df = key_dataframes['heart_rate']
intensity_df = key_dataframes['intensity']
dynamic_df = key_dataframes['dynamic']
single_heart_rate_df = key_dataframes['single_heart_rate']
single_spo2_df = key_dataframes['single_spo2']
training_load_df = key_dataframes['training_load']
single_stress_df = key_dataframes['single_stress']
stress_df = key_dataframes['stress']
watch_night_sleep_df = key_dataframes['watch_night_sleep']
resting_heart_rate_df = key_dataframes['resting_heart_rate']
watch_daytime_sleep_df = key_dataframes['watch_daytime_sleep']
weight_df = key_dataframes['weight']

# Intuition

I think we need to include control variables such as the number of hours sleep and the quality of sleep as this significantly influences your mental state after awaking.

I guess we can check whether someone has slept after their alarm right?

We must include variable for smart alarm or not. Can we find this in the settings from the mi band?

We are going to add features from our watch data to the behavioural data.

Features for our analysis:
- Average of the first 20 minutes after waking
- Whether the smart alarm actually woke us up in light sleep (not in deep or REM)

Hence we need:
- Waking time
- Heartrate data
- State when awaking

In [27]:
# make a temp to work with
temp_watch_night_sleep_df = watch_night_sleep_df.copy()
temp_watch_night_sleep_df.head()

,Uid,Sid,Key,Time,Value,UpdateTime,Person,sleep_deep_duration,items,wake_up_time,sleep_rem_duration,timezone,duration,sleep_awake_duration,bedtime,date_time,sleep_light_duration,awake_count
0,8210796956,huami.39873/AAAAC13TA75455,watch_night_sleep,1694649600,"{""sleep_deep_duration"":71,""items"":[{""start_tim...",1695116316,andy,71,"[{'start_time': 1694644980, 'state': 3, 'end_t...",1694668560,85,8,393,0,1694644980,1694649600,237,0
1,8210796956,huami.39873/AAAAC13TA75455,watch_night_sleep,1694736000,"{""awake_count"":0,""sleep_rem_duration"":48,""slee...",1695116316,andy,78,"[{'start_time': 1694729640, 'state': 3, 'end_t...",1694756520,48,8,448,0,1694729640,1694736000,322,0
2,8210796956,huami.39873/AAAAC13TA75455,watch_night_sleep,1694822400,"{""awake_count"":1,""sleep_rem_duration"":73,""slee...",1695116316,andy,76,"[{'start_time': 1694821560, 'state': 3, 'end_t...",1694850420,73,8,472,9,1694821560,1694822400,323,1
3,8210796956,huami.39873/AAAAC13TA75455,watch_night_sleep,1694908800,"{""awake_count"":0,""sleep_rem_duration"":90,""slee...",1695116316,andy,69,"[{'start_time': 1694906760, 'state': 3, 'end_t...",1694929800,90,8,384,0,1694906760,1694908800,225,0
4,8210796956,huami.39873/AAAAC13TA75455,watch_night_sleep,1694995200,"{""awake_count"":0,""sleep_rem_duration"":102,""sle...",1695116316,andy,34,"[{'start_time': 1694991660, 'state': 3, 'end_t...",1695015180,102,8,392,0,1694991660,1694995200,256,0


In [26]:
behaviour_tracking_data.head()

,Name,Drinks,Fastfood,Sports,Food 2h before sleep,Medication,Date created,Woke up by (smart) alarm,Woke up by external factors,Yesterday,Slept again after alarm
0,andy,False,True,True,False,False,2023-10-01T08:41:41.000Z,True,False,False,False
1,andy,False,True,False,False,False,2023-10-02T07:00:32.000Z,False,False,False,False
2,andy,False,False,False,False,False,2023-10-03T05:31:28.000Z,True,False,False,False
3,andy,False,False,False,False,False,2023-10-04T03:12:57.000Z,True,False,False,False
4,andy,False,False,False,False,False,2023-10-05T07:00:29.000Z,True,False,False,False


In [25]:
# cast to datetime for wake_up_time in temp_watch_daytime_sleep_df
import pandas as pd

temp_watch_night_sleep_df['wake_up_time'] = pd.to_datetime(temp_watch_night_sleep_df['wake_up_time'], unit='s')

# print a sentence mentioning who woke up when
for index, row in temp_watch_night_sleep_df.iterrows():
    #only print for tom
    if row['Person'] == 'tom':
        print(row['Person'], 'woke up at', row['wake_up_time'])



tom woke up at 2023-09-27 04:48:00
tom woke up at 2023-09-28 06:57:00
tom woke up at 2023-09-29 06:55:00
tom woke up at 2023-09-30 08:03:00
tom woke up at 2023-10-01 07:50:00
tom woke up at 2023-10-02 04:42:00
tom woke up at 2023-10-03 04:07:00
tom woke up at 2023-10-04 06:29:00
tom woke up at 2023-10-05 05:01:00
tom woke up at 2023-10-06 07:27:00
tom woke up at 2023-10-07 07:25:00
tom woke up at 2023-10-08 04:54:00
tom woke up at 2023-10-09 05:08:00
tom woke up at 2023-10-10 04:44:00
tom woke up at 2023-10-11 04:55:00
tom woke up at 2023-10-12 04:53:00
tom woke up at 2023-10-13 06:28:00
tom woke up at 2023-10-14 07:31:00
tom woke up at 2023-10-15 08:22:00
tom woke up at 2023-10-16 05:07:00
tom woke up at 2023-10-17 04:47:00
tom woke up at 2023-10-18 05:32:00
tom woke up at 2023-10-19 06:50:00
tom woke up at 2023-10-20 05:34:00
tom woke up at 2023-10-21 07:24:00
tom woke up at 2023-10-22 04:27:00
tom woke up at 2023-10-23 03:18:00
tom woke up at 2023-10-24 04:56:00
tom woke up at 2023-

Now we need to get save the wake time into the behavioural dataframe

### vanaf hier is vlm niet echt noodzakelijk

In [ ]:
master_frame[master_frame['Key'] == 'watch_daytime_sleep']['Value'].iloc[0]

In [ ]:
watch_daytime_sleep_df

In [ ]:
# Plot for all of us together
persons = heart_rate_df['Person'].unique()

plt.figure(figsize=(15, 8))

for person in persons:
    person_data = heart_rate_df[heart_rate_df['Person'] == person]
    plt.plot(person_data['Time'], person_data['bpm'], label=person)

plt.title('Heart Rate vs. Time')
plt.xlabel('Time')
plt.ylabel('Heart Rate (bpm)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plot for each person separately
for person in persons:
    person_data = heart_rate_df[heart_rate_df['Person'] == person]
    
    plt.figure(figsize=(10, 6))
    plt.plot(person_data['Time'], person_data['bpm'])
    
    plt.title(f'Heart Rate vs. Time for {person}')
    plt.xlabel('Time')
    plt.ylabel('Heart Rate (bpm)')
    plt.grid(True)
    
    plt.show()

In [ ]:
heart_rate_df['DateTime'] = pd.to_datetime(heart_rate_df['Time'], unit='s')
import matplotlib.dates as mdates

plt.figure(figsize=(15, 8))

# For each person, plot their data
for person in persons:
    person_data = heart_rate_df[heart_rate_df['Person'] == person]
    plt.plot(person_data['DateTime'], person_data['bpm'], label=person)

# Formatting the x-axis
ax = plt.gca()  # Get current axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.xticks(rotation=45)

plt.title('Heart Rate vs. Time')
plt.xlabel('Time')
plt.ylabel('Heart Rate (bpm)')
plt.tight_layout()  # To ensure labels fit
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
heart_rate_df['Hour'] = heart_rate_df['DateTime'].dt.hour
heart_rate_df['Minute'] = heart_rate_df['DateTime'].dt.minute
avg_heart_rate_per_person = heart_rate_df.groupby(['Person', 'Hour', 'Minute'])['bpm'].mean().reset_index()

avg_heart_rate_per_person['TimeOfDay'] = pd.to_datetime('2000-01-01 ' + avg_heart_rate_per_person['Hour'].astype(str) + ':' + avg_heart_rate_per_person['Minute'].astype(str))

plt.figure(figsize=(15, 8))

# For each person, plot their average heart rate
for person in persons:
    person_data = avg_heart_rate_per_person[avg_heart_rate_per_person['Person'] == person]
    plt.plot(person_data['TimeOfDay'], person_data['bpm'], label=person)

# Formatting the x-axis
ax = plt.gca()  # Get current axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.xticks(rotation=45)

plt.title('Average Heart Rate vs. Time of Day (by Person)')
plt.xlabel('Time of Day')
plt.ylabel('Average Heart Rate (bpm)')
plt.tight_layout()  # To ensure labels fit
plt.legend()
plt.grid(True)
plt.show()
